pip install psycopg2-binary
createdb -U postgres clients_db

In [ ]:
import psycopg2

def create_db(conn):
    with conn.cursor() as cur:
        cur.execute("""
        CREATE TABLE IF NOT EXISTS clients (
            client_id  SERIAL PRIMARY KEY,
            first_name VARCHAR(60) NOT NULL,
            last_name VARCHAR(60) NOT NULL,
            e_mail  VARCHAR(60) NOT NULL UNIQUE
            CHECK (e_mail ~ '^[a-zA-Z0-9.!#$%&''*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$')
            );
        """)
        cur.execute("""
        CREATE TABLE IF NOT EXISTS phones(
            phone_id SERIAL PRIMARY KEY,
            client_id INTEGER REFERENCES clients (client_id) ON DELETE CASCADE,
            phone_number TEXT NOT NULL UNIQUE
            CHECK (phone_number ~ '^\d{10}$')
            );
        """)
        conn.commit()
    return 'Базы данных созданы'

def add_client(conn, first_name, last_name, email, phone=None):
    with conn.cursor() as cur:
        if phone == None:
            cur.execute("""
                INSERT INTO clients(first_name, last_name, e_mail) VALUES
                    (%s, %s, %s);
                """, (first_name, last_name, email))
            d = 'Добавлен новый клиент, но он не указал телефон'

        else:
            cur.execute("""
                with rows as (
                INSERT INTO clients (first_name, last_name, e_mail) 
                VALUES (%s, %s, %s) RETURNING client_id 
                )
                INSERT INTO phones (client_id, phone_number) 
                SELECT client_id, %s
                FROM rows;
            """, (first_name, last_name, email, phone))
            d = 'Добавлен новый клиент. Телефон указан'
        conn.commit()
    return d

In [26]:
with psycopg2.connect(database="clients_db", user="postgres", password="fur2529") as conn:
    #print(create_db(conn))
    #print(add_client(conn, 'Том', 'Круз', 'tomcruser@gmail.com', '9826153947'))
    #print(add_client(conn, 'Дмитрий', 'Нагиев', 'dimadoma@gmail.com', '9826153946'))
    print(add_client(conn, 'Роберт', 'Де Ниро', 'deniroff@gmail.com'))
conn.close()

Новый клиент не указал телефон


  cur.execute("""
        INSERT INTO homework(number, description, course_id) VALUES(%s, %s, %s);
        """, (2, "задание посложнее", python_id))
        conn.commit()  # фиксируем в БД

import psycopg2

def create_db(conn):
    pass

def add_client(conn, first_name, last_name, email, phones=None):
    pass

def add_phone(conn, client_id, phone):
    pass

def change_client(conn, client_id, first_name=None, last_name=None, email=None, phones=None):
    pass

def delete_phone(conn, client_id, phone):
    pass

def delete_client(conn, client_id):
    pass

def find_client(conn, first_name=None, last_name=None, email=None, phone=None):
    pass


with psycopg2.connect(database="clients_db", user="postgres", password="postgres") as conn:
    pass  # вызывайте функции здесь

conn.close()